In [ ]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

jack = Customer('Park', 100)
jack

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        pass

class FidelityPromo(Promotion):
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

class LargeOrderPromo(Promotion):
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [ ]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5), LineItem('apple', 10, 1.5), LineItem('watermellon', 5, 5.0)]

In [ ]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [ ]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [ ]:
banana_cart = [LineItem('banana', 30, 0.5), LineItem('apple', 10, 1.5)]

In [ ]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [ ]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [ ]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [ ]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [19]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

def fidelity_promo(order):
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

In [20]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [21]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [22]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [23]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [24]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

In [25]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [26]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [27]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [28]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', 'jack', 'LineItem', 'Order', 'Promotion', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i2', 'joe', 'ann', 'cart', '_i3', '_3', '_i4', '_4', '_i5', 'banana_cart', '_i6', '_6', '_i7', 'long_order', '_i8', '_8', '_i9', '_9', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i20', '_20', '_i21', '_21', '_i22', '_22', '_i23', '_23', '_i24', 'promos', 'best_promo', '_i25', '_25', '_i26', '_26', '_i27', '_27', '_i28'])

In [29]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [30]:
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [31]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()

running inner()


In [32]:
target

<function __main__.deco.<locals>.inner()>

In [33]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x7a439ad9d000>)
running register(<function f2 at 0x7a439ad9cee0>)
running main()
registry -> [<function f1 at 0x7a439ad9d000>, <function f2 at 0x7a439ad9cee0>]
running f1()
running f2()
running f3()


In [34]:
promos = []

def promotion(promo_func):
    """promotion() decorator는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    """충성도 점수가 1000점 이상힌 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * 0.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

In [35]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [36]:
b=6
f1(3)

3
6


In [37]:
b = 6
def f2(a):
    print(a)
    print(b)
    b=9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [38]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [39]:
b

9

In [40]:
f3(3)

3
9


In [41]:
b=30
f3(3)

3
30


In [42]:
class Averager():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

In [43]:
avg = Averager()
avg(10)

10.0

In [44]:
avg(11)

10.5

In [45]:
avg(12)

11.0

In [46]:
def make_averager():
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)

  return averager

In [47]:
avg = make_averager()
avg(10)

10.0

In [48]:
avg(11)

10.5

In [49]:
avg(12)

11.0

In [50]:
avg.__code__.co_varnames

('new_value', 'total')

In [51]:
avg.__code__.co_freevars

('series',)

In [52]:
avg.__closure__

(<cell at 0x7a439b0937f0: list object at 0x7a439a7db180>,)

In [53]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [54]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    count += 1
    total += new_value
    return total / count

  return averager

In [55]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [56]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count

  return averager

In [57]:
avg = make_averager()
avg(10)

10.0

In [58]:
import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [59]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
  print('*' * 40, 'Calling snooze(.123)')
  snooze(.123)
  print('*' * 40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12323299s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000044s] factorial(1) -> 1
[0.00006171s] factorial(2) -> 2
[0.00085287s] factorial(3) -> 6
[0.00090316s] factorial(4) -> 24
[0.00092646s] factorial(5) -> 120
[0.00095992s] factorial(6) -> 720
6! = 720


In [60]:
factorial.__name__

'clocked'

In [61]:
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [64]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
  print('*' * 40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12328291s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000167s] factorial(1) -> 1
[0.00003791s] factorial(2) -> 2
[0.00006747s] factorial(3) -> 6
[0.00009680s] factorial(4) -> 24
[0.00012660s] factorial(5) -> 120
[0.00015497s] factorial(6) -> 720
6! = 720
